In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.        
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 3090


In [2]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [3]:
import json
with open('train.json', 'r') as f:
    data = json.load(f)

question_dic = {}
sparql_dic = {}

for x in range(4000):
    question_dic[x] = data[x]['corrected_question']
    sparql_dic[x] = data[x]['sparql_query']

In [4]:
with open('test.json', 'r') as f:
    data = json.load(f)

test_question_dic = {}
test_sparql_dic = {}

for x in range(len(data)):
    test_question_dic[x] = data[x]['corrected_question']
    test_sparql_dic[x] = data[x]['sparql_query']

In [5]:
def searchPredicatesWithoutType2(query):
    result = []

    li = query.split()
    index = 0
    while index < len(li):
        li[index] = li[index].strip()
        index += 1

    while '.' in li:
        dot_index = li.index('.')
        li.pop(dot_index)

    where_index = 0
    if 'where' in li:
        where_index = li.index('where')
    else:
        where_index = li.index('WHERE')

    if li[where_index + 1] == '{':
        next_predicate = where_index + 3
    else:
        next_predicate = where_index + 2

    while next_predicate < len(li):
        if 'http://dbpedia.org/ontology/' in li[next_predicate]:
            result.append(li[next_predicate][29:-1])
        elif 'http://dbpedia.org/property/' in li[next_predicate]:
            result.append(li[next_predicate][29:-1])

        next_predicate += 3

    return result

In [6]:
#build training and testing samples
def buildSamples(q_dic, s_dic):
    nlq = []
    predicates = []
    for x in range(len(q_dic)):
        current_predicates = searchPredicatesWithoutType2(s_dic[x])
        nlq.append(q_dic[x])
        predicates.append(current_predicates)
    return (nlq, predicates)

train_nlq, train_p = buildSamples(question_dic, sparql_dic)
test_nlq, test_p = buildSamples(test_question_dic, test_sparql_dic)

In [7]:
#check question and its correspoding predicates
q_id = 4
print(train_nlq[q_id])
print(train_p[q_id])

How many races have the horses bred by Jacques Van't Hart participated in?
['breeder', 'race']


In [8]:
# build a predicates-index dictionary
def build_labels_dic(p_list1, p_list2):
    res_dic = {}
    count = 0
    for x in p_list1:
        for y in x:
            if y not in res_dic:
                res_dic[y] = count
                count += 1
            
    for x in p_list2:
        for y in x:
            if y not in res_dic:
                res_dic[y] = count
                count += 1
            
    return res_dic

label_dic = build_labels_dic(train_p, test_p)

In [9]:
def build_one_to_one_samples(nlq, p):
    res = []
    for x in range(len(nlq)):
        for y in p[x]:
            res.append((nlq[x], y))
    return res

In [10]:
train_samples_pos = build_one_to_one_samples(train_nlq, train_p)

In [11]:
len(train_samples_pos)

6590

In [12]:
import random

def build_negative_samples(pos_samples):
    neg_rate = 4
    res = []
    for x in range(len(pos_samples)):
        for y in range(neg_rate):
            index = random.randint(0,len(pos_samples)-1)
            neg_sample = (pos_samples[x][0], pos_samples[index][1])
            while (neg_sample in pos_samples) or (neg_sample in res) :
                index = random.randint(0,len(pos_samples)-1)
                neg_sample = (pos_samples[x][0], pos_samples[index][1])
            res.append(neg_sample)
    return res

In [13]:
train_samples_neg = build_negative_samples(train_samples_pos)

In [14]:
len(train_samples_neg)

26360

In [15]:
train_data = train_samples_pos + train_samples_neg

In [16]:
train_labels = [[1]] * len(train_samples_pos) + [[0]] * len(train_samples_neg)

In [17]:
train_data[0]

('How many movies did Stanley Kubrick direct?', 'director')

# Tokenize Train/Test samples

In [18]:
train_embedding_pos = []
for x in train_data:
    train_embedding_pos.append([label_dic[x[1]]])

In [19]:
train_embedding_pos = torch.LongTensor(train_embedding_pos)

In [20]:
train_input_ids = []
train_attention_masks = []

for sample in train_data:
    sent = sample[0]
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    train_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    train_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_data[0][0])
print('Token IDs:', train_input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  How many movies did Stanley Kubrick direct?
Token IDs: tensor([  101,  2129,  2116,  5691,  2106,  6156, 13970, 25646,  3622,  1029,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [21]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(train_input_ids, train_attention_masks, train_embedding_pos, train_labels)
# Calculate the number of samples to include in each set.
train_size = int(1 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

32,950 training samples
    0 validation samples


In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

In [12]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
test_input_ids = []
test_attention_masks = []

# For every sentence...
for sent in test_nlq:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_labels = torch.tensor(test_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', test_nlq[0])
print('Token IDs:', test_input_ids[0])

Original:  Which architect of Marine Corps Air Station Kaneohe Bay was also tenant of New Sanno hotel /'
Token IDs: tensor([  101,  2029,  4944,  1997,  3884,  3650,  2250,  2276,  8472, 11631,
         2063,  3016,  2001,  2036, 16713,  1997,  2047,  2624,  3630,  3309,
         1013,  1005,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [13]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

3,600 training samples
  400 validation samples


In [14]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

test_size = len(test_dataset)
val_size = len(test_dataset) - test_size

test_dataset, val_dataset_not_important = random_split(test_dataset, [test_size, val_size])

print('{:>5,} testing samples'.format(test_size))
print('{:>5,} validation samples'.format(val_size))

1,000 testing samples
    0 validation samples


# Training：

In [23]:
from transformers import BertPreTrainedModel, AdamW, BertConfig, BertModel
import torch.nn as nn
class MyNET(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        #self.num_labels = config.num_labels
        self.rembedding = nn.Embedding(437,768)
        self.bert = BertModel(config)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        #Linear layers
        self.fc1 = nn.Linear(768 * 2, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        embedding_pos = None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]
        
        out = pooled_output
        embeddings = self.rembedding(embedding_pos)
        cat_layer = torch.cat((out, embeddings.view(-1,768)), dim=1)
        
        out = self.fc1(cat_layer)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        
        # Loss 计算部分检查？
        loss = None
        if labels is not None:
            loss_fct = nn.BCELoss()
            labels = labels.float()
            loss = loss_fct(out,labels)
        
        return (out, loss)

In [24]:
model = MyNET.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    #num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyNET: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing MyNET from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyNET from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyNET were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['rembedding.weight', 'fc1.weight', 'fc1.bias', 'f

MyNET(
  (rembedding): Embedding(437, 768)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [25]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [26]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [27]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [28]:
import random
import numpy as np

training_stats = []

total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_input_embedding_pos = batch[2].to(device)
        b_labels = batch[3].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             embedding_pos = b_input_embedding_pos,
                             labels=b_labels)
        
        logits = outputs[0]
        loss = outputs[1]

        total_train_loss += loss.item()
        
        

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        #print('current loss : ' + str(loss.item()))

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  2,060.    Elapsed: 0:00:09.
  Batch    80  of  2,060.    Elapsed: 0:00:19.
  Batch   120  of  2,060.    Elapsed: 0:00:29.
  Batch   160  of  2,060.    Elapsed: 0:00:38.
  Batch   200  of  2,060.    Elapsed: 0:00:48.
  Batch   240  of  2,060.    Elapsed: 0:00:57.
  Batch   280  of  2,060.    Elapsed: 0:01:06.
  Batch   320  of  2,060.    Elapsed: 0:01:16.
  Batch   360  of  2,060.    Elapsed: 0:01:25.
  Batch   400  of  2,060.    Elapsed: 0:01:34.
  Batch   440  of  2,060.    Elapsed: 0:01:43.
  Batch   480  of  2,060.    Elapsed: 0:01:53.
  Batch   520  of  2,060.    Elapsed: 0:02:02.
  Batch   560  of  2,060.    Elapsed: 0:02:12.
  Batch   600  of  2,060.    Elapsed: 0:02:21.
  Batch   640  of  2,060.    Elapsed: 0:02:30.
  Batch   680  of  2,060.    Elapsed: 0:02:40.
  Batch   720  of  2,060.    Elapsed: 0:02:49.
  Batch   760  of  2,060.    Elapsed: 0:02:59.
  Batch   800  of  2,060.    Elapsed: 0:03:08.
  Batch   840  of

  Batch   600  of  2,060.    Elapsed: 0:02:21.
  Batch   640  of  2,060.    Elapsed: 0:02:31.
  Batch   680  of  2,060.    Elapsed: 0:02:41.
  Batch   720  of  2,060.    Elapsed: 0:02:50.
  Batch   760  of  2,060.    Elapsed: 0:03:00.
  Batch   800  of  2,060.    Elapsed: 0:03:09.
  Batch   840  of  2,060.    Elapsed: 0:03:19.
  Batch   880  of  2,060.    Elapsed: 0:03:28.
  Batch   920  of  2,060.    Elapsed: 0:03:38.
  Batch   960  of  2,060.    Elapsed: 0:03:46.
  Batch 1,000  of  2,060.    Elapsed: 0:03:56.
  Batch 1,040  of  2,060.    Elapsed: 0:04:05.
  Batch 1,080  of  2,060.    Elapsed: 0:04:14.
  Batch 1,120  of  2,060.    Elapsed: 0:04:24.
  Batch 1,160  of  2,060.    Elapsed: 0:04:33.
  Batch 1,200  of  2,060.    Elapsed: 0:04:42.
  Batch 1,240  of  2,060.    Elapsed: 0:04:51.
  Batch 1,280  of  2,060.    Elapsed: 0:05:01.
  Batch 1,320  of  2,060.    Elapsed: 0:05:10.
  Batch 1,360  of  2,060.    Elapsed: 0:05:20.
  Batch 1,400  of  2,060.    Elapsed: 0:05:30.
  Batch 1,440

# 下面的代码是测试，不用管

In [26]:
import random
import numpy as np

training_stats = []

total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        logits = outputs[1]
        loss = outputs[0]

        total_train_loss += loss.item()
        
        

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        #print('current loss : ' + str(loss.item()))

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    
    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_recall = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        logits = outputs[1]
        loss = outputs[0]
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        eval_acc, eval_recall = flat_accuracy(logits, b_labels)
        
        total_eval_accuracy += eval_acc
        total_eval_recall += eval_recall
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_recall = total_eval_recall / len(validation_dataloader)
    print("  Accuracy: {0:.8f}".format(avg_val_accuracy))
    print("  Recall: {0:.8f}".format(avg_val_recall))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


======== Epoch 1 / 20 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:09.
  Batch    80  of    225.    Elapsed: 0:00:19.
  Batch   120  of    225.    Elapsed: 0:00:28.
  Batch   160  of    225.    Elapsed: 0:00:38.
  Batch   200  of    225.    Elapsed: 0:00:47.

  Average training loss: 0.56
  Training epcoh took: 0:00:52

Running Validation...
  Accuracy: 0.00321429
  Recall: 0.01401833
  Validation Loss: 0.43
  Validation took: 0:00:01

======== Epoch 2 / 20 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:09.
  Batch    80  of    225.    Elapsed: 0:00:18.
  Batch   120  of    225.    Elapsed: 0:00:28.
  Batch   160  of    225.    Elapsed: 0:00:37.
  Batch   200  of    225.    Elapsed: 0:00:47.

  Average training loss: 0.34
  Training epcoh took: 0:00:53

Running Validation...
  Accuracy: 0.00464286
  Recall: 0.02066668
  Validation Loss: 0.26
  Validation took: 0:00:01

======== Epoch 3 / 20 ========
Training...
  Batch    40  of    225.    Elapsed: 

  Batch    40  of    225.    Elapsed: 0:00:09.
  Batch    80  of    225.    Elapsed: 0:00:19.
  Batch   120  of    225.    Elapsed: 0:00:27.
  Batch   160  of    225.    Elapsed: 0:00:36.
  Batch   200  of    225.    Elapsed: 0:00:46.

  Average training loss: 0.03
  Training epcoh took: 0:00:51

Running Validation...
  Accuracy: 0.00642857
  Recall: 0.02844570
  Validation Loss: 0.03
  Validation took: 0:00:01

======== Epoch 20 / 20 ========
Training...
  Batch    40  of    225.    Elapsed: 0:00:09.
  Batch    80  of    225.    Elapsed: 0:00:19.
  Batch   120  of    225.    Elapsed: 0:00:28.
  Batch   160  of    225.    Elapsed: 0:00:38.
  Batch   200  of    225.    Elapsed: 0:00:47.

  Average training loss: 0.03
  Training epcoh took: 0:00:53

Running Validation...
  Accuracy: 0.00642857
  Recall: 0.02844570
  Validation Loss: 0.03
  Validation took: 0:00:01


In [27]:
model.eval()

total_eval_accuracy = 0
total_eval_recall = 0

for step, batch in enumerate(test_dataloader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():        


        seq_result = model(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=b_input_mask,
                          labels = b_labels)

        logits = seq_result[1]
        loss = seq_result[0]

        #print(logits[1].sigmoid())
        #print(b_labels[1])


        #target = labels.cpu().detach().numpy().tolist()
        #output = np.array(torch.sigmoid(logits).cpu().detach().numpy().tolist()) >= th
        #output = np.array(torch.sigmoid(logits).cpu().detach().numpy().tolist())


        acc, recall = flat_accuracy(logits, b_labels)
        #print(acc)
        #print(recall)
        total_eval_accuracy += acc
        total_eval_recall += recall
        
print(total_eval_accuracy / len(test_dataloader))
print(total_eval_recall / len(test_dataloader))

0.002692743764172335
0.012203546316846808


In [28]:
model.eval()

total_eval_accuracy = 0
total_eval_recall = 0

for step, batch in enumerate(train_dataloader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():        


        seq_result = model(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=b_input_mask,
                          labels = b_labels)

        logits = seq_result[1]
        loss = seq_result[0]

        #print(logits[1].sigmoid())
        #print(b_labels[1])


        #target = labels.cpu().detach().numpy().tolist()
        #output = np.array(torch.sigmoid(logits).cpu().detach().numpy().tolist()) >= th
        #output = np.array(torch.sigmoid(logits).cpu().detach().numpy().tolist())


        acc, recall = flat_accuracy(logits, b_labels)
        #print(acc)
        #print(recall)
        total_eval_accuracy += acc
        total_eval_recall += recall
        
print(total_eval_accuracy / len(test_dataloader))
print(total_eval_recall / len(test_dataloader))

0.023667800453514715
0.10885886995962916


In [13]:
q_id = 1
print(train_nlq[q_id])
print(train_input_ids[q_id])

Which city's foundeer is John Forbes?
tensor([  101,  2029,  2103,  1005,  1055,  2179, 11510,  2003,  2198, 10822,
         1029,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [58]:
train_input_ids = train_input_ids.to('cuda')
train_attention_masks = train_attention_masks.to('cuda')
print(train_input_ids[0].shape)
print(train_attention_masks[0].shape)

torch.Size([64])
torch.Size([64])


In [84]:
model = MyNET.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    #num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyNET: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing MyNET from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyNET from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyNET were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['rembedding.weight', 'fc1.weight', 'fc1.bias', 'f

MyNET(
  (rembedding): Embedding(437, 768)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [85]:
input_emb = torch.LongTensor([[1],[2]])
input_emb = input_emb.to('cuda')

In [86]:
b_input_ids = train_input_ids[0:2]
b_input_mask = train_attention_masks[0:2]

model.eval()
with torch.no_grad():        
    seq_result = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,embedding_pos = input_emb, labels = target)
    print(seq_result)

(tensor([[0.5074],
        [0.5070]], device='cuda:0'), tensor(0.6788, device='cuda:0'))


In [75]:
print(seq_result[0].shape)

torch.Size([2, 1])


In [80]:
target = torch.tensor([[1],[1]])
target.shape

torch.Size([2, 1])

In [81]:
target = target.to('cuda')